In [6]:
import numpy as np
import pandas as pd
import pickle
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import dataholders
import utils


path = 'data/data.pickle'

temp_path = 'data/raw/uft_flight07.mat'
wind_path = 'data/raw/actos_flight07.mat'

temp_path2 = 'data/raw/uft_flight16.mat'
wind_path2 = 'data/raw/actos_flight16.mat'

In [2]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
import keras.backend as K

Using TensorFlow backend.


In [3]:
from sklearn.model_selection import train_test_split

In [45]:
N = 101

In [46]:
holder = dataholders.TempWindData(temp_path, wind_path)

mean = holder.T1.mean()
std = holder.T1.std()


rolled = utils.rolling_window(holder.T1, N)

X_train = rolled[:-1]

Y_train = holder.T1[N:]

X_train = (X_train - mean) / std
Y_train = (Y_train - mean) / std

In [47]:
holder = dataholders.TempWindData(temp_path2, wind_path2)

#mean2 = holder2.T1.mean()
#std2 = holder2.T1.std()

rolled = utils.rolling_window(holder.T1, N)

X_test = rolled[:-1]

Y_test = holder.T1[N:]

X_test = (X_test - mean) / std
Y_test = (Y_test - mean) / std

In [110]:
model = Sequential()
model.add(Dense(100, activation='tanh', input_shape=(N,)))
model.add(Dense(50, activation='tanh'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(10, activation='tanh'))
model.add(Dense(1))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_70 (Dense)             (None, 100)               10200     
_________________________________________________________________
dense_71 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_72 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_73 (Dense)             (None, 20)                1020      
_________________________________________________________________
dense_74 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_75 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_76 (Dense)             (None, 1)                 11        
Total para

In [111]:
model.compile('adam', loss='mse', metrics=['mae'])

In [112]:
model.fit(X_train, Y_train, batch_size=10000, epochs=20)

Epoch 1/20
663467/663467 [==============================] - 2s - loss: 0.1021 - mean_absolute_error: 0.1655     
Epoch 2/20
663467/663467 [==============================] - 2s - loss: 0.0053 - mean_absolute_error: 0.0287     
Epoch 3/20
663467/663467 [==============================] - 2s - loss: 0.0027 - mean_absolute_error: 0.0206     
Epoch 4/20
663467/663467 [==============================] - 2s - loss: 0.0019 - mean_absolute_error: 0.0201     
Epoch 5/20
663467/663467 [==============================] - 2s - loss: 0.0015 - mean_absolute_error: 0.0194     
Epoch 6/20
663467/663467 [==============================] - 2s - loss: 0.0013 - mean_absolute_error: 0.0200     
Epoch 7/20
663467/663467 [==============================] - 2s - loss: 0.0011 - mean_absolute_error: 0.0179     
Epoch 8/20
663467/663467 [==============================] - 2s - loss: 8.6679e-04 - mean_absolute_error: 0.0160     
Epoch 9/20
663467/663467 [==============================] - 2s - loss: 9.1414e-04 - mean_abs

In [99]:
model.evaluate(X_train, Y_train, batch_size=1024)

661504/663467 [============================>.] - ETA: 0s

[0.3226852812150604, 0.47332050827447419]

In [100]:
model.evaluate(X_test, Y_test, batch_size=1024)

777216/791758 [============================>.] - ETA: 0s

[0.1839318829997264, 0.36480586540774917]

In [113]:
preds_train = model.predict(X_train, batch_size=1024).ravel()

In [114]:
preds_test = model.predict(X_test, batch_size=1024).ravel()

In [115]:
plt.rcParams['figure.figsize'] = [20,8]

plt.plot(preds_train*std + mean, c='b')
plt.plot(Y_train*std + mean, c='g')
plt.show()

In [97]:
plt.rcParams['figure.figsize'] = [20,8]

plt.plot(preds_test*std + mean, c='b')
plt.plot(Y_test*std + mean, c='g')
plt.show()

In [55]:
abs((Y_test) - (preds_test)).max()

0.67197766568919848

In [31]:
(Y2 > preds).mean()

0.87044333979954736

In [27]:
model.evaluate(X2, Y2, batch_size=1024)

774144/791808 [============================>.] - ETA: 0s

[0.006274319926910331, 0.00018319686503334104]

# Ideas:

0. Predict the filtered series

0. Use T2, lwc, sonic(1/2/3)